## Introduction

We'll be setting up a motion sensor camera, primarily for keeping an eye on a pet. The project will be organized into several steps:
<br>
Introduction <br>
Setting up the raspberry pi <br>
SSHing into the raspberry pi <br>
Emailing Your IP address on boot <br>
Installing and setting up Motion <br>
Emailing Videos from Motion on detection <br>


### What you'll need: 
* A Raspberry Pi 3 model b ~\\$35 
* 8gb Micro SD card ~\\$10 
* USB Webcam ~\\$20 
* micro usb cable ~\\$5 
* USB wall adapter ~\\$5 
* A WiFi connection 


### What you'll need access to for setup:
* A monitor <br>
* A mouse and keyboard <br>
* A computer with an SD card slot <br>
* A Micro SD to SD card adapter <br>

## Let's setup the Raspberry Pi
Once you have the raspberry pi, follow the steps here (https://projects.raspberrypi.org/en/projects/noobs-install) to install raspbian onto the micro sd card.

### Connecting to WiFi
First you will need to connect the device to WiFi. This may have been done already by clicking the WiFi Network icon in the menu bar of your raspberry pi if using a display. If you are using a headless device, follow the steps here (https://learn.adafruit.com/adafruits-raspberry-pi-lesson-3-network-setup/setting-up-wifi-with-occidentalis) to connect to WiFi via CLI.

### Using SSH
We'll take a look at enabling SSH to allow you to configure and edit the device without a monitor, just mouse and keyboard.

## Accessing raspberry pi via SSH
### Enable SSH
* Open a command line on the device and type `sudo raspi-config`

You will now see the configuration menu. Use the arrow, tab, and enter keys to navigate to the first into interfacing options, then to enable SSH, as well as the camera, then exit and reboot the pi.

Next, you'll need to find your Pi's IP address. In a command terminal, type:

`sudo ifconfig`

And your IP address should be in the wlan0 section that pops up, right under Link encap:ethernet.

Last, you'll need to either open up the built in terminal app on a mac, or find and follow instructions for a 3rd party ssh app for windows. In the Terminal app, type:

`ssh pi@YOUR IP ADDRES`

If you haven't changed the password while in Raspi-config, then your password to enter when prompted will be the default password: raspberry.

From there, you should be good to go!

## Email IP Address on Boot
In this step we will focus on how to access your Raspberry Pi, regardless of the network you are on. Depending on the network, the Pi's IP address changes. And if you do not have a monitor setup, you would need to ssh into the Pi to trigger the motion detection protocol, change motion preferences, or access anything else on the device. To solve this problem we will write a python script that emails us the Pi's IP current IP address upon start up. The python script is as follows and was stored in an directory marked "background".

`#start in home directory`
`cd ~`

`#create background folder
`mkdir background`

`#create python script
`sudo nano emailip.py`

`#write in emailip.py
`import socket
`s = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
`s.connect(("8.8.8.8", 80))
`print(s.getsockname()[0])x = s.getsockname()[0]s.close()`

`import smtplib
`from email.MIMEMultipart import MIMEMultipart
`from email.MIMEText import MIMEText
`fromaddr = 'YOUR ADDRESS'
`toaddr = 'RECEIVING ADDRESS'`

`msg = MIMEMultipart()`

`msg['From'] = fromaddr
`msg['To'] = toaddr
`msg['Subject'] = "IP Address"`

`body = xmsg.attach(MIMEText(body, 'plain'))`

`server = smtplib.SMTP('smtp.gmail.com', 587)
`server.starttls()
`server.login(fromaddr, 'YOUR PASSWORD')`

`text = msg.as_string()
server.sendmail(fromaddr, toaddr, text)
server.quit()`

`#then this does it on reboot
sudo nano /etc/rc.local`

`#enter in /etc/rc.local
while ! /sbin/ifconfig wlan0 | grep -q 'inet addr:[0-9]';
do sleep 3
done`

`_IP=\\$(hostname -I) || true
if [ "\\$_IP" ]; then
printf "My IP address is %s\n" "$_IP"
python /home/pi/Background/emailip.py &
fi
exit 0`

`#and you're done`

## Installing Motion
`#update pi
sudo apt-get update
sudo apt-get upgrade`

`#download
sudo apt-get install motion`

`#now edit this file with the following changes
sudo nano /etc/motion/motion.conf`

`#to set a standard for this tutorial, change
#################
daemon on
stream_localhost off
webcontrol_localhost off
ffmpeg_output_movies on
target_dir /var/lib/motion`

`##################
#local web browser streaming options
##################`

`stream_maxrate 100 #This will allow for real-time streaming but requires more bandwidth & resources
framerate 60 #This will allow for 60 frames to be captured per second #the higher this gets, the slower the video processing
width 640 #This changes the width of the image displayed
height 480 #This changes the height of the image displayed`

`##################
#emailing video settings in .../motion.conf`
##################

`#delete the ' ; ' in front of the line, the semicolon comments the line out
on_event_start python /home/pi/background/motionalert.py %f
on_movie_end python /home/pi/background/motionvid.py %f`

`##################
#aesthetics
##################`

`#choices described in the file
output_pictures locate_motion_style`

`##################`

`#then change
sudo nano /etc/default/motion`

`#to
start_motion_daemon=yes`

`#last, start the B**** up
sudo service motion start`

`#you can change the command to 'stop', or 'restart'`

## Emailing Motion Detected Video
Email when motion is detected:

`#start at home`
`dircd ~`

`#create motion alert python script`
`sudo nano /home/pi/background/motionalert.py`

`#write` <br>
`import smtplib` <br>
`from datetime import datetime` <br>
`from email.MIMEMultipart import MIMEMultipart` <br>
`from email.MIMEText import MIMEText` <br>
`fromaddr = "YOURADDRESS"` <br>
`toaddr = "RECIEVINGADDRESS"` <br>
 <br>
`msg = MIMEMultipart()` <br>
`msg['From'] = fromaddr` <br>
`msg['To'] = toaddr` <br>
`msg['Subject'] = "Motion Detected"` <br>
`body = 'A motion has been detected.\nTime: %s' % str(datetime.now())` <br>
`msg.attach(MIMEText(body, 'plain'))` <br>
`server = smtplib.SMTP('smtp.gmail.com', 587)` <br>
`server.starttls()` <br>
`server.login(fromaddr, "YOURPASSWORD")` <br>
`text = msg.as_string()` <br>
`server.sendmail(fromaddr, toaddr, text)` <br>
`server.quit()` <br>
 <br>
Email Video of motion when video is saved: <br>
 <br>
`#start at home` <br>
`dircd ~` <br>
 <br>
`#create motion video python script` <br>
`sudo nano /home/pi/background/motionvid.py` <br>
 <br>
`import smtplib` <br>
`from email.MIMEMultipart import MIMEMultipart` <br>
`from email.MIMEText import MIMEText` <br>
`from email.MIMEBase import MIMEBase` <br>
`from email import encoders` <br>
 <br>

`fromaddr = "YOUR EMAIL"` <br>
`toaddr = "EMAIL ADDRESS YOU SEND TO"` <br>
`msg = MIMEMultipart()` <br>
`msg['From'] = fromaddr` <br>
`msg['To'] = toaddr` <br>
`msg['Subject'] = "Motion Cam Activated"` <br>
`body = "Video of Motion Detected"` <br>
`msg.attach(MIMEText(body, 'plain'))` <br>
`import os` <br>

`rootpath = '/var/lib/motion'` <br>
`filelist = [os.path.join(rootpath, f) for f in os.listdir(rootpath)]` <br>
`filelist = [f for f in filelist if os.path.isfile(f)]` <br>
`newest = max(filelist, key=lambda x: os.stat(x).st_mtime)` <br>
`filename = newest` <br>
 <br>
`import os` <br>
`rootpath = '/var/lib/motion'` <br>
`filelist = [os.path.join(rootpath, f) for f in os.listdir(rootpath)]` <br>
`filelist = [f for f in filelist if os.path.isfile(f)]` <br>
`newest = max(filelist, key=lambda x: os.stat(x).st_mtime)` <br>
 <br>
`attachment = open(newest, "rb")` <br>
`part = MIMEBase('application', 'octet-stream')` <br>
`part.set_payload((attachment).read())` <br>
`encoders.encode_base64(part)` <br>
`part.add_header('Content-Disposition', "attachment; filename= %s" % filename)` <br>
`msg.attach(part)` <br>
 <br>
`server = smtplib.SMTP('smtp.gmail.com', 587)` <br>
`server.starttls()` <br>
`server.login(fromaddr, "YOUR PASSWORD")` <br>
`text = msg.as_string()` <br>
`server.sendmail(fromaddr, toaddr, text)` <br>
`server.quit()` <br>